In [1]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="korquad-v1",
    downstream_model_dir="/scratch/qualis/nlp/checkpoint-qa",
    downstream_corpus_root_dir="/scratch/qualis/nlp",
    max_seq_length=128,
    max_query_length=32,
    doc_stride=64,
    fp16=True,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=5,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

2023-06-10 13:30:44.164294: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 13:30:53.609467: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-10 13:30:58.559285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/cuda/11.7/lib64:/scratch/qualis/miniconda3/lib:/scratch/qualis/mecab/lib:/scratch/qualis/TensorRT-7.2.3.4/lib::
2023-06-10 13:30:58.599828: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] C

In [2]:
args

QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/scratch/qualis/nlp', downstream_model_dir='/scratch/qualis/nlp/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=32, fp16=True, tpu_cores=0, tqdm_enabled=True)

In [3]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [4]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/scratch/qualis/nlp', downstream_model_dir='/scratch/qualis/nlp/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=32, fp16=True, tpu_cores=0, tqdm_enabled=True)


In [5]:
nlpbook.download_downstream_dataset(args)

Downloading: 38.5MB [00:01, 19.8MB/s]                            
Downloading: 3.88MB [00:00, 4.83MB/s]                          


In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)


In [7]:
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
corpus = KorQuADV1Corpus()
train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from train dataset file at /scratch/qualis/nlp/korquad-v1
convert squad examples to features: 100%|██████████| 57688/57688 [00:55<00:00, 1044.47it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? [SEP] 18 ##3 ##9년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽고 그 내용 ##에 마음이 끌려 이를 소재 ##로 해서 하나의 교 ##향 ##곡 ##을 쓰 ##려는 뜻을 갖 ##는다 . 이 시기 바 ##그 ##너 ##는 18 ##3 ##8년 ##에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 걲 ##은 상황이 ##라 좌 ##절 ##과 실망 ##에 가득 ##했 ##으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만나는 파 ##우스 ##트 ##의 심 ##경 ##에 공감 ##했다고 한다 . 또한 파리 ##에서 아 ##브 ##네 ##크 ##의 지휘 ##로 파리 음악 ##원 관 ##현 ##악 ##단이 연 ##주 ##하는 베 ##토 [SEP]
INFO:ratsnlp:answer: 교 ##향 ##곡
INFO:ratsnlp:features: QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 1

In [8]:
train_dataset[:2]

[QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 10966, 11728, 12023, 14657, 4091, 8598, 16639, 341, 4573, 4771, 4027, 2139, 8478, 14416, 214, 8202, 17, 2451, 13007, 1480, 4313, 4538, 4008, 8601, 4633, 22903, 4113, 1676, 868, 4913, 7965, 1789, 4203, 4110, 15031, 786, 250, 4057, 10878, 4007, 2593, 4094, 4128, 10289, 4113, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
len(train_dataset[0].input_ids)

128

In [10]:
print(tokenizer.convert_ids_to_tokens(train_dataset[0].input_ids))
print(tokenizer.convert_ids_to_tokens(train_dataset[0].input_ids)[45:48])
print(tokenizer.convert_ids_to_tokens(train_dataset[0].input_ids)[18+64]) # doc sliding = 64

['[CLS]', '바', '##그', '##너', '##는', '괴', '##테', '##의', '파', '##우스', '##트를', '읽고', '무엇을', '쓰고', '##자', '했', '##는가', '?', '[SEP]', '18', '##3', '##9년', '바', '##그', '##너', '##는', '괴', '##테', '##의', '파', '##우스', '##트', '##을', '처음', '읽고', '그', '내용', '##에', '마음이', '끌려', '이를', '소재', '##로', '해서', '하나의', '교', '##향', '##곡', '##을', '쓰', '##려는', '뜻을', '갖', '##는다', '.', '이', '시기', '바', '##그', '##너', '##는', '18', '##3', '##8년', '##에', '빛', '독', '##촉', '##으로', '산', '##전', '##수', '##전을', '다', '걲', '##은', '상황이', '##라', '좌', '##절', '##과', '실망', '##에', '가득', '##했', '##으며', '메', '##피', '##스', '##토', '##펠', '##레스', '##를', '만나는', '파', '##우스', '##트', '##의', '심', '##경', '##에', '공감', '##했다고', '한다', '.', '또한', '파리', '##에서', '아', '##브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관', '##현', '##악', '##단이', '연', '##주', '##하는', '베', '##토', '[SEP]']
['교', '##향', '##곡']
##에


In [11]:
print(tokenizer.convert_ids_to_tokens(train_dataset[1].input_ids))

['[CLS]', '바', '##그', '##너', '##는', '괴', '##테', '##의', '파', '##우스', '##트를', '읽고', '무엇을', '쓰고', '##자', '했', '##는가', '?', '[SEP]', '가득', '##했', '##으며', '메', '##피', '##스', '##토', '##펠', '##레스', '##를', '만나는', '파', '##우스', '##트', '##의', '심', '##경', '##에', '공감', '##했다고', '한다', '.', '또한', '파리', '##에서', '아', '##브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관', '##현', '##악', '##단이', '연', '##주', '##하는', '베', '##토', '##벤', '##의', '교', '##향', '##곡', '9', '##번', '##을', '듣고', '깊은', '감', '##명을', '받았는데', ',', '이것이', '이', '##듬', '##해', '1월', '##에', '파', '##우스', '##트', '##의', '서', '##곡', '##으로', '쓰여', '##진', '이', '작품', '##에', '조금이라도', '영향을', '끼', '##쳤', '##으리', '##라는', '것은', '의심', '##할', '여지가', '없다', '.', '여기', '##의', '라', '##단', '##조', '조성', '##의', '경우', '##에도', '그의', '전기', '##에', '적', '##혀', '있는', '것처럼', '단순한', '정신적', '피로', '##나', '[SEP]']


In [12]:
print(tokenizer.convert_ids_to_tokens(train_dataset[2].input_ids))

['[CLS]', '바', '##그', '##너', '##는', '괴', '##테', '##의', '파', '##우스', '##트를', '읽고', '무엇을', '쓰고', '##자', '했', '##는가', '?', '[SEP]', '파', '##우스', '##트', '##의', '서', '##곡', '##으로', '쓰여', '##진', '이', '작품', '##에', '조금이라도', '영향을', '끼', '##쳤', '##으리', '##라는', '것은', '의심', '##할', '여지가', '없다', '.', '여기', '##의', '라', '##단', '##조', '조성', '##의', '경우', '##에도', '그의', '전기', '##에', '적', '##혀', '있는', '것처럼', '단순한', '정신적', '피로', '##나', '실', '##의가', '반영', '##된', '것이', '아니라', '베', '##토', '##벤', '##의', '합', '##창', '##교', '##향', '##곡', '조성', '##의', '영향을', '받은', '것을', '볼', '수', '있다', '.', '그렇게', '교', '##향', '##곡', '작', '##곡', '##을', '18', '##3', '##9년', '##부터', '40년', '##에', '걸', '##쳐', '파리', '##에서', '착', '##수', '##했', '##으나', '1', '##악', '##장을', '쓴', '뒤에', '중단', '##했다', '.', '또한', '작품', '##의', '완성', '##과', '동시에', '그는', '이', '서', '##곡', '(', '[SEP]']


In [13]:
print(train_dataset[2:5])

[QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 3231, 23243, 4104, 4042, 1843, 4771, 7965, 28987, 4153, 2451, 15489, 4113, 13928, 17283, 575, 4261, 26783, 8114, 8852, 9107, 4082, 28498, 8131, 17, 8225, 4042, 1114, 4281, 4194, 17138, 4042, 9961, 8222, 14041, 10892, 4113, 2524, 4443, 8032, 12710, 21602, 18625, 24569, 4136, 2009, 11012, 17068, 4130, 8544, 8141, 1543, 4775, 4170, 4042, 3367, 4183, 4267, 4573, 4771, 17138, 4042, 17283, 9838, 9153, 1576, 1931, 8120, 17, 8117, 341, 4573, 4771, 2478, 4771, 4027, 8601, 4633, 29697, 8042, 25305, 4113, 254, 4133, 23008, 7971, 2840, 4110, 4062, 10410, 20, 4158, 8915, 2141, 11726, 13014, 8258, 17, 10384, 15489, 4042, 16520, 4128, 15831, 20048, 2451, 1843, 4771, 11, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [14]:
print(tokenizer.convert_ids_to_tokens(train_dataset[4].input_ids))

['[CLS]', '바', '##그', '##너', '##는', '괴', '##테', '##의', '파', '##우스', '##트를', '읽고', '무엇을', '쓰고', '##자', '했', '##는가', '?', '[SEP]', '##으나', ',', '실제로는', '이루어', '##지지', '##는', '않았다', '.', '결국', '초', '##연', '##은', '4년', '반', '##이', '지난', '후에', '드', '##레스', '##덴', '##에서', '연', '##주', '##되었고', '재', '##연', '##도', '이루어', '##졌지', '##만', ',', '이후에', '그대로', '방치', '##되고', '말았', '##다', '.', '그', '사이에', '그는', '리', '##엔', '##치', '##와', '방', '##황', '##하는', '네', '##덜', '##란드', '##인을', '완성', '##하고', '탄', '##호', '##이', '##저', '##에도', '착', '##수', '##하는', '등', '분', '##주', '##한', '시간을', '보냈', '##는데', ',', '그런', '바쁜', '생활', '##이', '이', '곡', '##을', '잊', '##게', '한', '것이', '아닌가', '하는', '의견', '##도', '있다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [15]:
train_dataset[5]

QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 341, 4573, 4771, 2478, 4771, 4027, 13655, 2141, 11726, 13014, 4062, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 10966, 11728, 12023, 14657, 4091, 8598, 16639, 341, 4573, 4771, 4027, 2139, 8478, 14416, 214, 8202, 17, 2451, 13007, 1480, 4313, 4538, 4008, 8601, 4633, 22903, 4113, 1676, 868, 4913, 7965, 1789, 4203, 4110, 15031, 786, 250, 4057, 10878, 4007, 2593, 4094, 4128, 10289, 4113, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [16]:
print(tokenizer.convert_ids_to_tokens(train_dataset[5].input_ids))

['[CLS]', '바', '##그', '##너', '##는', '교', '##향', '##곡', '작', '##곡', '##을', '어디까지', '쓴', '뒤에', '중단', '##했', '##는가', '?', '[SEP]', '18', '##3', '##9년', '바', '##그', '##너', '##는', '괴', '##테', '##의', '파', '##우스', '##트', '##을', '처음', '읽고', '그', '내용', '##에', '마음이', '끌려', '이를', '소재', '##로', '해서', '하나의', '교', '##향', '##곡', '##을', '쓰', '##려는', '뜻을', '갖', '##는다', '.', '이', '시기', '바', '##그', '##너', '##는', '18', '##3', '##8년', '##에', '빛', '독', '##촉', '##으로', '산', '##전', '##수', '##전을', '다', '걲', '##은', '상황이', '##라', '좌', '##절', '##과', '실망', '##에', '가득', '##했', '##으며', '메', '##피', '##스', '##토', '##펠', '##레스', '##를', '만나는', '파', '##우스', '##트', '##의', '심', '##경', '##에', '공감', '##했다고', '한다', '.', '또한', '파리', '##에서', '아', '##브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관', '##현', '##악', '##단이', '연', '##주', '##하는', '베', '##토', '[SEP]']


In [17]:
len(train_dataset)

223875

In [18]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

/scratch/qualis/miniconda3/envs/nlp-py310/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [19]:
len(train_dataloader)

6997

In [20]:
#from torch.utils.data import SequentialSampler
val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from val dataset file at /scratch/qualis/nlp/korquad-v1
convert squad examples to features: 100%|██████████| 5533/5533 [00:05<00:00, 1028.47it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 198 ##9년 6월 30 ##일 평양 ##축 ##전에 대표로 파견 된 인물 ##은 ? [SEP] 198 ##9년 2월 15일 여의도 농민 폭력 시위를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처벌 ##에 ##관 ##한 ##법 ##률 ##위반 ) 으로 지명 ##수 ##배 ##되었다 . 198 ##9년 3월 12 ##일 서울 ##지 ##방 ##검찰 ##청 공안 ##부는 임종석 ##의 사전 ##구속 ##영장 ##을 발부 ##받았 ##다 . 같은 해 6월 30 ##일 평양 ##축 ##전에 임 ##수 ##경을 대표로 파견 ##하여 국가보안법 ##위반 혐의가 추가 ##되었다 . 경찰은 12월 18 ##일 ~ 20 ##일 사이 서울 경 ##희 ##대학 ##교 ##에서 임종석 ##이 성 ##명 발표 ##를 추진 ##하고 있다는 첩 ##보를 입 ##수 ##했고 , 12월 18 ##일 오전 7시 40 ##분 경 가스 ##총 ##과 [SEP]
INFO:ratsnlp:answer: 임 ##수 ##경을
INFO:ratsnlp:features: QAFeatures(input_ids=[2, 25704, 29697, 23343, 8476, 4046, 14601, 4728, 8409, 24345, 20158, 902, 10393, 4057, 32, 3, 25704, 29697, 18163, 26757, 20604, 29804, 10901, 25099, 13225, 4047, 16704, 11, 10901, 12175, 4086, 11417, 4113, 

In [21]:
len(val_dataset)
val_dataset

In [22]:
from transformers import BertConfig, BertForQuestionAnswering
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)
model = BertForQuestionAnswering.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model c

In [23]:
print(model)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [24]:
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)

In [25]:
trainer = nlpbook.get_trainer_org(args)

Multiprocessing is handled by SLURM.
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [26]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/scratch/qualis/miniconda3/envs/nlp-py310/lib/python3.10/site-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(

  | Name  | Type                     | Params
---------------------------------------------------
0 | model | BertForQuestionAnswering | 108 M 
---------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
216.659   Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# Model Testing

In [27]:
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/scratch/qualis/nlp/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: /scratch/qualis/nlp/checkpoint-qa/epoch=0-val_loss=0.46.ckpt


In [28]:
print(args.pretrained_model_name)
print(args.downstream_model_checkpoint_fpath)

beomi/kcbert-base
/scratch/qualis/nlp/checkpoint-qa/epoch=0-val_loss=0.46.ckpt


In [29]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [30]:
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cuda"),
    #map_location=torch.device("cpu"),
)

In [31]:
#fine_tuned_model_ckpt

In [32]:
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

In [33]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering(pretrained_model_config)

In [35]:
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [36]:
def inference_fn(question, context):
    if question and context:
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
       )
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.max_seq_length,
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

In [37]:
context = "직통 쾌속은 2008년 3월 15일의 시간표 개정으로 평일 아침 출근 시간대에 신설된 오사카 순환선 직통 열차이다. 정차역은 한와 선 내에서는 쾌속과 동일하며 오사카 순환선 안에서는 각역에 정차한다. 차량은 223계를 사용하고 있다. 상행은 10편이 오사카 순환선 외선을 경유해 오사카 역·교바시 방면으로 운전된다. 오사카 순환선을 일주하는 열차 등과 같이 도중역으로서 일단 덴노지 역에 정차하는 것에서 오사카 방면으로 운행하지 않는다는 오해를 피하기 위해 교바시 행을 제외하고 오사카 역까지는 오사카 순환선으로 안내하고 있다. 하행은 오사카 순환선 덴노지 역에서 교바시 역을 경유해 오토리 역이나 와카야마 역 행이 각각 1편씩 운행하고 있다."

In [38]:
question = "직통 쾌속이 생겨난 해는?"

In [39]:
inference_fn(question, context)

{'question': '직통 쾌속이 생겨난 해는?',
 'context': '직통 쾌속은 2008년 3월 15일의 시간표 개정으로 평일 아침 출근 시간대에 신설된 오사카 순환선 직통 열차이다. 정차역은 한와 선 내에서는 쾌속과 동일하며 오사카 순환선 안에서는 각역에 정차한다. 차량은 223계를 사용하고 있다. 상행은 10편이 오사카 순환선 외선을 경유해 오사카 역·교바시 방면으로 운전된다. 오사카 순환선을 일주하는 열차 등과 같이 도중역으로서 일단 덴노지 역에 정차하는 것에서 오사카 방면으로 운행하지 않는다는 오해를 피하기 위해 교바시 행을 제외하고 오사카 역까지는 오사카 순환선으로 안내하고 있다. 하행은 오사카 순환선 덴노지 역에서 교바시 역을 경유해 오토리 역이나 와카야마 역 행이 각각 1편씩 운행하고 있다.',
 'answer': '##은 한와 선 내에서는 쾌속과 동일하며 오사카 순환선 안에서는 각역에 정차한다. 차량은 223계를 사용하고 있다. 상행은 10편이 오사카 순환선 외선을 경유해 오사카 역 · 교바시 방면으로 운전된다. 오사카 순환선을 일주하는 열차 등과 같이 도중역으로서 일단 덴'}

In [40]:
truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
       )

In [41]:
inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.max_seq_length,
            return_token_type_ids=True,
        )

In [42]:
inputs

{'input_ids': [2, 2689, 4149, 3059, 16808, 13762, 4030, 3376, 4008, 32, 3, 2689, 4149, 3059, 4474, 4057, 9227, 22903, 18306, 26757, 4042, 8485, 4302, 14540, 7965, 3288, 4046, 10830, 13490, 8485, 10883, 28538, 4130, 28623, 1933, 4185, 4124, 2689, 4149, 2275, 16059, 4020, 17, 2539, 4495, 4119, 4057, 3354, 4196, 1846, 609, 8652, 3059, 4474, 4128, 12829, 8481, 28623, 1933, 4185, 4124, 16771, 4008, 198, 4119, 4113, 2539, 4495, 8008, 17, 13747, 4057, 15190, 4633, 11984, 29978, 8120, 17, 1801, 4080, 4057, 8240, 15464, 28623, 1933, 4185, 4124, 2339, 10909, 16073, 4032, 28623, 2270, 97, 341, 4054, 4039, 1497, 26209, 9381, 8210, 17, 28623, 1933, 4185, 10909, 12668, 7966, 2275, 4495, 963, 4128, 8386, 867, 4276, 4119, 10166, 9083, 849, 4041, 4102, 2270, 4113, 2539, 4495, 7966, 14688, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [43]:
outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})

In [44]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 0.4417,  0.1925,  0.4811,  0.4708,  0.0286,  0.3127, -0.0036,  0.1152,
         -0.2720,  0.1406,  0.1467, -0.4801,  0.3757,  0.1022, -0.3915,  0.3087,
         -0.4444,  0.1712, -0.1085,  0.2849,  0.2038, -0.1493,  0.1230, -0.6527,
          0.3427,  0.0778,  0.6245,  0.0849,  0.1778, -0.3275, -0.3602,  0.0736,
          0.5815, -0.2287,  0.1379, -0.2697, -0.6289, -0.5460, -0.0679,  0.1733,
         -0.2476, -0.0249, -0.0267,  0.5080,  0.0415,  0.2354,  0.7151,  0.1569,
         -0.2820, -0.0428, -0.1896,  0.0063,  0.3874, -0.7396,  0.5704,  0.4891,
         -0.3000, -0.3170,  0.2584, -0.0103, -0.2485,  0.1057, -0.4400, -0.1116,
          0.1147, -0.8206,  0.4234, -0.6029, -0.2217,  0.2806,  0.2207,  0.1841,
          0.1332,  0.4258,  0.1357, -0.4735,  0.1759,  0.0406, -0.3344, -0.4455,
          0.2387,  0.3354,  0.0371, -0.3036,  0.1778,  0.3516, -0.8035, -0.4581,
          0.2130, -0.1133, -0.7639,  0.1793,  0.1073,  0

In [45]:
outputs.start_logits.shape

torch.Size([1, 128])

In [46]:
start_pred = outputs.start_logits.argmax(dim=-1).item()

In [47]:
start_pred

46

In [48]:
end_pred = outputs.end_logits.argmax(dim=-1).item()
end_pred

118

In [49]:
pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])

In [50]:
pred_text

'##은 한와 선 내에서는 쾌속과 동일하며 오사카 순환선 안에서는 각역에 정차한다. 차량은 223계를 사용하고 있다. 상행은 10편이 오사카 순환선 외선을 경유해 오사카 역 · 교바시 방면으로 운전된다. 오사카 순환선을 일주하는 열차 등과 같이 도중역으로서 일단 덴'